# Import packages

In [1]:
import pandas as pd

# from stopwordsiso import stopwords
# from text_cleaner.processor.common import SYMBOLS_AND_PUNCTUATION_EXTENSION, GENERAL_PUNCTUATION
# from text_cleaner.processor.chinese import CHINESE_SYMBOLS_AND_PUNCTUATION

# from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer
# from sklearn.linear_model import LogisticRegression

from sklearn import metrics

/home/yueliu/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Sentence transformers

## Load models

In [2]:
model = SentenceTransformer(model_name_or_path="Classical/Yinka", trust_remote_code=True)

d:\projects\SMU\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Informational data

In [6]:
data_train = pd.read_csv("D:\projects\SMU\LLM\dataset_622\informatical_train.csv", encoding="utf-8")
data_val = pd.read_csv("D:\projects\SMU\LLM\dataset_622\informatical_val.csv", encoding="utf-8")
data_test = pd.read_csv("D:\projects\SMU\LLM\dataset_622\informatical_test.csv", encoding="utf-8")

In [7]:
data_train.head(10)

,text,label
0,他克莫司软膏（普特彼） 0.1%*10g 数量：1盒,1
1,家附近医院心内科先做好,0
2,出院小结能否拍照给我？,0
3,呃，有可能的，这个小朋友就这几个最常见的突然会加重的因素，脸上加重的话，这个食物如果身上还好...,1
4,我倾向良性概率大,1
5,结节性质不能确定，建议右肺穿刺活检明确诊断；如果为肿瘤，做PET/CT评估疾病程度,1
6,性价比很高啊。,0
7,没关系的。,0
8,对的，要巩固,0
9,好的，不客气,0


In [12]:
stopwords_list = stopwords("zh")

clean_text = []
for i, row in data_train.iterrows():
    text_data = str(row["text"])

    for word in stopwords_list:
        clean_text_data = text_data.replace(word, '')

    clean_text_data = SYMBOLS_AND_PUNCTUATION_EXTENSION.remove(clean_text_data)
    clean_text_data = GENERAL_PUNCTUATION.remove(clean_text_data)
    clean_text_data = CHINESE_SYMBOLS_AND_PUNCTUATION.remove(clean_text_data)   
    clean_text_data = ''.join(clean_text_data.split())

    clean_text.append(clean_text_data)

In [13]:
data_train.insert(len(data_train.columns), "clean_text", clean_text)
data_train.head(10)

,text,label,clean_text
0,他克莫司软膏（普特彼） 0.1%*10g 数量：1盒,1,他克莫司软膏普特彼0110g数量1盒
1,家附近医院心内科先做好,0,家附近医院心内科先做好
2,出院小结能否拍照给我？,0,出院小结能否拍照给我
3,呃，有可能的，这个小朋友就这几个最常见的突然会加重的因素，脸上加重的话，这个食物如果身上还好...,1,呃有可能的这个小朋友就这几个最常见的突然会加重的因素脸上加重的话这个食物如果身上还好那么脸上...
4,我倾向良性概率大,1,我倾向良性概率大
5,结节性质不能确定，建议右肺穿刺活检明确诊断；如果为肿瘤，做PET/CT评估疾病程度,1,结节性质不能确定建议右肺穿刺活检明确诊断如果为肿瘤做PETCT评估疾病程度
6,性价比很高啊。,0,性价比很高啊
7,没关系的。,0,没关系的
8,对的，要巩固,0,对的要巩固
9,好的，不客气,0,好的不客气


In [20]:
data_train['embeddings'] = data_train['clean_text'].apply(model.encode)

In [21]:
data_train.head(10)

,text,label,clean_text,embeddings
0,他克莫司软膏（普特彼） 0.1%*10g 数量：1盒,1,他克莫司软膏普特彼0110g数量1盒,"[-0.09918186, -1.1277616, -0.98428404, 0.30943..."
1,家附近医院心内科先做好,0,家附近医院心内科先做好,"[-1.4207051, -1.1746099, 0.26470053, 1.0255177..."
2,出院小结能否拍照给我？,0,出院小结能否拍照给我,"[-0.52355343, -0.96749634, -0.20458244, 1.0927..."
3,呃，有可能的，这个小朋友就这几个最常见的突然会加重的因素，脸上加重的话，这个食物如果身上还好...,1,呃有可能的这个小朋友就这几个最常见的突然会加重的因素脸上加重的话这个食物如果身上还好那么脸上...,"[-0.029868163, 0.026902512, -0.80432284, 1.228..."
4,我倾向良性概率大,1,我倾向良性概率大,"[-1.4161913, -0.14997435, -0.541139, 1.1043158..."
5,结节性质不能确定，建议右肺穿刺活检明确诊断；如果为肿瘤，做PET/CT评估疾病程度,1,结节性质不能确定建议右肺穿刺活检明确诊断如果为肿瘤做PETCT评估疾病程度,"[-1.0413795, -0.66310954, 0.14560078, 1.058798..."
6,性价比很高啊。,0,性价比很高啊,"[-1.1063848, -0.9463874, 0.035060763, 0.410800..."
7,没关系的。,0,没关系的,"[-0.7566406, -1.3819497, -1.118149, 0.8126394,..."
8,对的，要巩固,0,对的要巩固,"[-0.6767457, -1.7042383, -0.79112196, 0.912259..."
9,好的，不客气,0,好的不客气,"[-0.56192386, -1.1051131, -0.94136465, 1.19099..."


In [22]:
data_train.to_csv("./data/new_df_with_embeddings.csv", index=False)

In [25]:
X_train = data_train['embeddings'].to_list()
y_train = data_train['label'].to_list()

In [35]:
LR = LogisticRegression(max_iter=1000)
LR.fit(X_train,y_train)

LogisticRegression(max_iter=1000)

In [28]:
stopwords_list = stopwords("zh")

clean_text = []
for i, row in data_val.iterrows():
    text_data = str(row["text"])

    for word in stopwords_list:
        clean_text_data = text_data.replace(word, '')

    clean_text_data = SYMBOLS_AND_PUNCTUATION_EXTENSION.remove(clean_text_data)
    clean_text_data = GENERAL_PUNCTUATION.remove(clean_text_data)
    clean_text_data = CHINESE_SYMBOLS_AND_PUNCTUATION.remove(clean_text_data)   
    clean_text_data = ''.join(clean_text_data.split())

    clean_text.append(clean_text_data)

In [29]:
data_val.insert(len(data_val.columns), "clean_text", clean_text)
data_val.head(10)

,text,label,clean_text
0,治疗费用一般一块一元硬币大小的面积1000元/次，第一次多500元开机费。,1,治疗费用一般一块一元硬币大小的面积1000元次第一次多500元开机费
1,不排除这个可能 所以要经常拍片子 一般三个月拍一次,1,不排除这个可能所以要经常拍片子一般三个月拍一次
2,当然，在刺激性方面，0.1的也比0.03的要强一点，但是这个刺激性用个几天一周也就耐受了这个...,1,当然在刺激性方面01的也比003的要强一点但是这个刺激性用个几天一周也就耐受了这个没有没有什...
3,术后短期内有刺激性干咳是正常的，注意多活动,1,术后短期内有刺激性干咳是正常的注意多活动
4,、如无明显不适，可继续用原药。,1,如无明显不适可继续用原药
5,其他活动可以照常,0,其他活动可以照常
6,可考虑手术,1,可考虑手术
7,建议你口服芬吗通治疗,1,建议你口服芬吗通治疗
8,如果初次发现，强烈建议三个月后复查，比对前后变化,1,如果初次发现强烈建议三个月后复查比对前后变化
9,3.手指蜕皮不要去撕，也不要去抠，不要弄破，不适当的撕或者抠容易加重皮肤受损。,1,3手指蜕皮不要去撕也不要去抠不要弄破不适当的撕或者抠容易加重皮肤受损


In [30]:
data_val['embeddings'] = data_val['clean_text'].apply(model.encode)

In [31]:
X_val = data_val['embeddings'].to_list()
y_val = data_val["label"].to_list()

In [34]:
data_val.to_csv("./data/new_df_with_embeddings_val.csv", index=False)

In [36]:
predicted = LR.predict(X_val)
print("Logistic Regression Accuracy:", metrics.accuracy_score(y_val, predicted))
print("Logistic Regression Precision:", metrics.precision_score(y_val, predicted))
print("Logistic Regression Recall:", metrics.recall_score(y_val, predicted))
print("Logistic Regression F1_score:", metrics.f1_score(y_val, predicted))

Logistic Regression Accuracy: 0.8866943866943867
Logistic Regression Precision: 0.9212730318257957
Logistic Regression Recall: 0.8986928104575164
Logistic Regression F1_score: 0.9098428453267163


## Emotional data

In [16]:
data_train = pd.read_csv("D:\projects\SMU\LLM\dataset_622\emotional_train.csv", encoding="utf-8")
data_val = pd.read_csv("D:\projects\SMU\LLM\dataset_622\emotional_val.csv", encoding="utf-8")
data_test = pd.read_csv("D:\projects\SMU\LLM\dataset_622\emotional_test.csv", encoding="utf-8")

In [21]:
stopwords_list = stopwords("zh")

clean_text = []
for i, row in data_train.iterrows():
    text_data = str(row["text"])

    for word in stopwords_list:
        clean_text_data = text_data.replace(word, '')

    clean_text_data = SYMBOLS_AND_PUNCTUATION_EXTENSION.remove(clean_text_data)
    clean_text_data = GENERAL_PUNCTUATION.remove(clean_text_data)
    clean_text_data = CHINESE_SYMBOLS_AND_PUNCTUATION.remove(clean_text_data)   
    clean_text_data = ''.join(clean_text_data.split())

    clean_text.append(clean_text_data)

In [22]:
data_train.insert(len(data_train.columns), "clean_text", clean_text)
data_train.head(10)

,text,label,clean_text
0,他克莫司软膏（普特彼） 0.1%*10g 数量：1盒,0,他克莫司软膏普特彼0110g数量1盒
1,家附近医院心内科先做好,0,家附近医院心内科先做好
2,出院小结能否拍照给我？,0,出院小结能否拍照给我
3,呃，有可能的，这个小朋友就这几个最常见的突然会加重的因素，脸上加重的话，这个食物如果身上还好...,0,呃有可能的这个小朋友就这几个最常见的突然会加重的因素脸上加重的话这个食物如果身上还好那么脸上...
4,我倾向良性概率大,0,我倾向良性概率大
5,结节性质不能确定，建议右肺穿刺活检明确诊断；如果为肿瘤，做PET/CT评估疾病程度,0,结节性质不能确定建议右肺穿刺活检明确诊断如果为肿瘤做PETCT评估疾病程度
6,性价比很高啊。,0,性价比很高啊
7,没关系的。,0,没关系的
8,对的，要巩固,0,对的要巩固
9,好的，不客气,0,好的不客气


In [19]:
stopwords_list = stopwords("zh")

clean_text = []
for i, row in data_val.iterrows():
    text_data = str(row["text"])

    for word in stopwords_list:
        clean_text_data = text_data.replace(word, '')

    clean_text_data = SYMBOLS_AND_PUNCTUATION_EXTENSION.remove(clean_text_data)
    clean_text_data = GENERAL_PUNCTUATION.remove(clean_text_data)
    clean_text_data = CHINESE_SYMBOLS_AND_PUNCTUATION.remove(clean_text_data)   
    clean_text_data = ''.join(clean_text_data.split())

    clean_text.append(clean_text_data)

In [20]:
data_val.insert(len(data_val.columns), "clean_text", clean_text)
data_val.head(10)

,text,label,clean_text
0,治疗费用一般一块一元硬币大小的面积1000元/次，第一次多500元开机费。,0,治疗费用一般一块一元硬币大小的面积1000元次第一次多500元开机费
1,不排除这个可能 所以要经常拍片子 一般三个月拍一次,0,不排除这个可能所以要经常拍片子一般三个月拍一次
2,当然，在刺激性方面，0.1的也比0.03的要强一点，但是这个刺激性用个几天一周也就耐受了这个...,0,当然在刺激性方面01的也比003的要强一点但是这个刺激性用个几天一周也就耐受了这个没有没有什...
3,术后短期内有刺激性干咳是正常的，注意多活动,0,术后短期内有刺激性干咳是正常的注意多活动
4,、如无明显不适，可继续用原药。,0,如无明显不适可继续用原药
5,其他活动可以照常,0,其他活动可以照常
6,可考虑手术,0,可考虑手术
7,建议你口服芬吗通治疗,0,建议你口服芬吗通治疗
8,如果初次发现，强烈建议三个月后复查，比对前后变化,0,如果初次发现强烈建议三个月后复查比对前后变化
9,3.手指蜕皮不要去撕，也不要去抠，不要弄破，不适当的撕或者抠容易加重皮肤受损。,0,3手指蜕皮不要去撕也不要去抠不要弄破不适当的撕或者抠容易加重皮肤受损


In [23]:
data_train['embeddings'] = data_train['clean_text'].apply(model.encode)

In [24]:
data_val['embeddings'] = data_val['clean_text'].apply(model.encode)

In [25]:
X_train = data_train['embeddings'].to_list()
y_train = data_train['label'].to_list()

In [26]:
LR = LogisticRegression(max_iter=1000)
LR.fit(X_train,y_train)

LogisticRegression(max_iter=1000)

In [27]:
X_val = data_val['embeddings'].to_list()
y_val = data_val["label"].to_list()

In [28]:
predicted = LR.predict(X_val)
print("Logistic Regression Accuracy:", metrics.accuracy_score(y_val, predicted))
print("Logistic Regression Precision:", metrics.precision_score(y_val, predicted))
print("Logistic Regression Recall:", metrics.recall_score(y_val, predicted))
print("Logistic Regression F1_score:", metrics.f1_score(y_val, predicted))

Logistic Regression Accuracy: 0.9726264726264726
Logistic Regression Precision: 0.2911392405063291
Logistic Regression Recall: 0.5
Logistic Regression F1_score: 0.368


# LLMs

## Zero-shot learning - Qwen2-7B - Single prompt

### Load data and model

In [3]:
informational_data_train = pd.read_csv("C:\SMU\LLM\data\informatical_train.csv", encoding="utf-8")
emotional_data_train = pd.read_csv("C:\SMU\LLM\data\emotional_train.csv", encoding="utf-8")

In [1]:
device = "cpu" # the device to load the model onto

# Now you do not need to add "trust_remote_code=True"
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-7B-Instruct",
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-7B-Instruct")

c:\SMU\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\pvtnguyet\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\pvtnguyet\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  Fi

In [25]:
# Try with one sample
informational_sample = informational_data_train.iloc[0]["text"]
neither_sample = informational_data_train.iloc[1]["text"]
emotional_sample = emotional_data_train[emotional_data_train["label"] == 1].iloc[0]["text"]

In [26]:
prompt = f"""Act as a discerning analyzer with expertise in both medical communication and emotional intelligence. Given a physician's online reply, I need you to analyze the content and context of the message. Based on your analysis, choose the best tag in four tags below that describe it:
A. Informational Support
B. Emotional Support
C. Both informational and emotional support
D. Neither informational support or emotional support

Informational support should be considered as providing advice, facts, or guidance that can be used to solve a problem or address a question.
Emotional support should be considered as responses that offer empathy, concern, encouragement, reassurance, or aim to share the patient's happiness or sadness and to express caring and concern. 

For a response to be tagged as both, it must clearly contain elements of both informational and emotional support. If the reply lacks elements of both types of support, it should be tagged as neither. 

It's crucial that you focus solely on the content and the conveyed support type in the physician's reply to categorize it accurately. Please return only the chosen label.

The online physician's reply for analysis:
{informational_sample}
"""
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

# Directly use generate() and tokenizer.decode() to get the output.
# Use `max_new_tokens` to control the maximum output length.
generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [27]:
response

'A'

In [42]:
prompt = f"""Act as a discerning analyzer with expertise in both medical communication and emotional intelligence. Given a physician's online reply, I need you to analyze the content and context of the message. Based on your analysis, choose the best tag in four tags below that describe it:
A. Informational Support
B. Emotional Support
C. Both informational and emotional support
D. Neither informational support or emotional support

Informational support should be considered as providing advice, facts, or guidance that can be used to solve a problem or address a question.
Emotional support should be considered as responses that offer empathy, concern, encouragement, reassurance, or aim to share the patient's happiness or sadness and to express caring and concern. 

For a response to be tagged as both, it must clearly contain elements of both informational and emotional support. If the reply lacks elements of both types of support, it should be tagged as neither. 

It's crucial that you focus solely on the content and the conveyed support type in the physician's reply to categorize it accurately. Please return only the chosen label.

The online physician's reply for analysis:
{emotional_sample}
"""
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

# Directly use generate() and tokenizer.decode() to get the output.
# Use `max_new_tokens` to control the maximum output length.
generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [43]:
response

'C. Both informational and emotional support'

In [44]:
emotional_sample

'图片看上去是天疱疮，现在的口腔溃疡情况也是天疱疮引起的，口苦的症状是可能和药物有关，不要紧的'

### Inference

In [47]:
# loop through 21 samples (0-20)
informational_results = []
for i, row in informational_data_train[informational_data_train["label"] == 1].iloc[:21].iterrows():
    informational_sample = row["text"]
    prompt = f"""Act as a discerning analyzer with expertise in both medical communication and emotional intelligence. Given a physician's online reply, I need you to analyze the content and context of the message. Based on your analysis, choose the best tag in four tags below that describe it:
    A. Informational Support
    B. Emotional Support
    C. Both informational and emotional support
    D. Neither informational support or emotional support

    Informational support should be considered as providing advice, facts, or guidance that can be used to solve a problem or address a question.
    Emotional support should be considered as responses that offer empathy, concern, encouragement, reassurance, or aim to share the patient's happiness or sadness and to express caring and concern. 

    For a response to be tagged as both, it must clearly contain elements of both informational and emotional support. If the reply lacks elements of both types of support, it should be tagged as neither. 

    It's crucial that you focus solely on the content and the conveyed support type in the physician's reply to categorize it accurately. Please return only the chosen label.

    The online physician's reply for analysis:
    {informational_sample}
    """
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    # Directly use generate() and tokenizer.decode() to get the output.
    # Use `max_new_tokens` to control the maximum output length.
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    informational_results.append(response)

In [49]:
# loop through 21 samples (0-20)
emotional_results = []
for i, row in emotional_data_train[emotional_data_train["label"] == 1].iloc[:21].iterrows():
    emotional_sample = row["text"]
    prompt = f"""Act as a discerning analyzer with expertise in both medical communication and emotional intelligence. Given a physician's online reply, I need you to analyze the content and context of the message. Based on your analysis, choose the best tag in four tags below that describe it:
    A. Informational Support
    B. Emotional Support
    C. Both informational and emotional support
    D. Neither informational support or emotional support

    Informational support should be considered as providing advice, facts, or guidance that can be used to solve a problem or address a question.
    Emotional support should be considered as responses that offer empathy, concern, encouragement, reassurance, or aim to share the patient's happiness or sadness and to express caring and concern. 

    For a response to be tagged as both, it must clearly contain elements of both informational and emotional support. If the reply lacks elements of both types of support, it should be tagged as neither. 

    It's crucial that you focus solely on the content and the conveyed support type in the physician's reply to categorize it accurately. Please return only the chosen label.

    The online physician's reply for analysis:
    {emotional_sample}
    """
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    # Directly use generate() and tokenizer.decode() to get the output.
    # Use `max_new_tokens` to control the maximum output length.
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    emotional_results.append(response)

In [48]:
# loop through 21 samples (0-20)
non_informational_results = []
for i, row in informational_data_train[informational_data_train["label"] == 0].iloc[:21].iterrows():
    non_informational_sample = row["text"]
    prompt = f"""Act as a discerning analyzer with expertise in both medical communication and emotional intelligence. Given a physician's online reply, I need you to analyze the content and context of the message. Based on your analysis, choose the best tag in four tags below that describe it:
    A. Informational Support
    B. Emotional Support
    C. Both informational and emotional support
    D. Neither informational support or emotional support

    Informational support should be considered as providing advice, facts, or guidance that can be used to solve a problem or address a question.
    Emotional support should be considered as responses that offer empathy, concern, encouragement, reassurance, or aim to share the patient's happiness or sadness and to express caring and concern. 

    For a response to be tagged as both, it must clearly contain elements of both informational and emotional support. If the reply lacks elements of both types of support, it should be tagged as neither. 

    It's crucial that you focus solely on the content and the conveyed support type in the physician's reply to categorize it accurately. Please return only the chosen label.

    The online physician's reply for analysis:
    {non_informational_sample}
    """
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    # Directly use generate() and tokenizer.decode() to get the output.
    # Use `max_new_tokens` to control the maximum output length.
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    non_informational_results.append(response)

In [56]:
# loop through 21 samples (0-20)
non_emotional_results = []
for i, row in emotional_data_train[emotional_data_train["label"] == 0].iloc[:21].iterrows():
    non_emotional_sample = row["text"]
    prompt = f"""Act as a discerning analyzer with expertise in both medical communication and emotional intelligence. Given a physician's online reply, I need you to analyze the content and context of the message. Based on your analysis, choose the best tag in four tags below that describe it:
    A. Informational Support
    B. Emotional Support
    C. Both informational and emotional support
    D. Neither informational support or emotional support

    Informational support should be considered as providing advice, facts, or guidance that can be used to solve a problem or address a question.
    Emotional support should be considered as responses that offer empathy, concern, encouragement, reassurance, or aim to share the patient's happiness or sadness and to express caring and concern. 

    For a response to be tagged as both, it must clearly contain elements of both informational and emotional support. If the reply lacks elements of both types of support, it should be tagged as neither. 

    It's crucial that you focus solely on the content and the conveyed support type in the physician's reply to categorize it accurately. Please return only the chosen label.

    The online physician's reply for analysis:
    {non_emotional_sample}
    """
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    # Directly use generate() and tokenizer.decode() to get the output.
    # Use `max_new_tokens` to control the maximum output length.
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    non_emotional_results.append(response)

In [55]:
mini_batch_informational = informational_data_train[informational_data_train["label"] == 1].iloc[:21]
mini_batch_informational["predicted_label"] = informational_results
mini_batch_informational

,text,label,predicted_label
0,他克莫司软膏（普特彼） 0.1%*10g 数量：1盒,1,D. Neither informational support or emotional ...
3,呃，有可能的，这个小朋友就这几个最常见的突然会加重的因素，脸上加重的话，这个食物如果身上还好...,1,C. Both informational and emotional support
4,我倾向良性概率大,1,D. Neither informational support or emotional ...
5,结节性质不能确定，建议右肺穿刺活检明确诊断；如果为肿瘤，做PET/CT评估疾病程度,1,C. Both informational and emotional support
11,卵巢功能LH/FSH略高，睾酮不高，主要的治疗方法是减轻体重，控制好体重，内分泌就会改善,1,C. Both informational and emotional support
13,也就是说没有一个检查办法能够确诊，药物过敏。,1,D. Neither informational support or emotional ...
14,你心理负担比较重，那就做一下吧，应该没有什么问题的，做完后把结果给我看一下吧,1,C. Both informational and emotional support
16,建议阴道塞药，目前治疗HPV感染的药物很多。,1,A. Informational Support
17,这个是大头痱子,1,D. Neither informational support or emotional ...
18,他也是高的啊，那么我觉得这个还是要考虑这个呃，她是一个标志物啊，也就是肿瘤有进展的这个可能性...,1,C. Both informational and emotional support


In [59]:
normalize_informational_predictions = [1 if ( predict.startswith("A.") or predict.startswith("C.")) else 0 for predict in informational_results]
mini_batch_informational["normalize_predicted_label"] = normalize_informational_predictions
mini_batch_informational

,text,label,predicted_label,normalize_predicted_label
0,他克莫司软膏（普特彼） 0.1%*10g 数量：1盒,1,D. Neither informational support or emotional ...,0
3,呃，有可能的，这个小朋友就这几个最常见的突然会加重的因素，脸上加重的话，这个食物如果身上还好...,1,C. Both informational and emotional support,1
4,我倾向良性概率大,1,D. Neither informational support or emotional ...,0
5,结节性质不能确定，建议右肺穿刺活检明确诊断；如果为肿瘤，做PET/CT评估疾病程度,1,C. Both informational and emotional support,1
11,卵巢功能LH/FSH略高，睾酮不高，主要的治疗方法是减轻体重，控制好体重，内分泌就会改善,1,C. Both informational and emotional support,1
13,也就是说没有一个检查办法能够确诊，药物过敏。,1,D. Neither informational support or emotional ...,0
14,你心理负担比较重，那就做一下吧，应该没有什么问题的，做完后把结果给我看一下吧,1,C. Both informational and emotional support,1
16,建议阴道塞药，目前治疗HPV感染的药物很多。,1,A. Informational Support,1
17,这个是大头痱子,1,D. Neither informational support or emotional ...,0
18,他也是高的啊，那么我觉得这个还是要考虑这个呃，她是一个标志物啊，也就是肿瘤有进展的这个可能性...,1,C. Both informational and emotional support,1


In [53]:
mini_batch_non_informational = informational_data_train[informational_data_train["label"] == 0].iloc[:21]
mini_batch_non_informational["predicted_label"] = non_informational_results
mini_batch_non_informational

,text,label,predicted_label
1,家附近医院心内科先做好,0,A. Informational Support
2,出院小结能否拍照给我？,0,D. Neither informational support or emotional ...
6,性价比很高啊。,0,D. Neither informational support or emotional ...
7,没关系的。,0,D. Neither informational support or emotional ...
8,对的，要巩固,0,D. Neither informational support or emotional ...
9,好的，不客气,0,D. Neither informational support or emotional ...
10,谈不上。,0,D. Neither informational support or emotional ...
12,床位紧张，我们尽量吧,0,D. Neither informational support or emotional ...
15,是的。,0,D. Neither informational support or emotional ...
19,中医科医生知道的。,0,D. Neither informational support or emotional ...


In [60]:
normalize_non_informational_predictions = [1 if ( predict.startswith("A.") or predict.startswith("C.")) else 0 for predict in non_informational_results]
mini_batch_non_informational["normalize_predicted_label"] = normalize_non_informational_predictions
mini_batch_non_informational

,text,label,predicted_label,normalize_predicted_label
1,家附近医院心内科先做好,0,A. Informational Support,1
2,出院小结能否拍照给我？,0,D. Neither informational support or emotional ...,0
6,性价比很高啊。,0,D. Neither informational support or emotional ...,0
7,没关系的。,0,D. Neither informational support or emotional ...,0
8,对的，要巩固,0,D. Neither informational support or emotional ...,0
9,好的，不客气,0,D. Neither informational support or emotional ...,0
10,谈不上。,0,D. Neither informational support or emotional ...,0
12,床位紧张，我们尽量吧,0,D. Neither informational support or emotional ...,0
15,是的。,0,D. Neither informational support or emotional ...,0
19,中医科医生知道的。,0,D. Neither informational support or emotional ...,0


In [54]:
mini_batch_emotional = emotional_data_train[emotional_data_train["label"] == 1].iloc[:21]
mini_batch_emotional["predicted_label"] = emotional_results
mini_batch_emotional

,text,label,predicted_label
56,图片看上去是天疱疮，现在的口腔溃疡情况也是天疱疮引起的，口苦的症状是可能和药物有关，不要紧的,1,C. Both informational and emotional support
66,你做得很好。,1,D. Neither informational support or emotional ...
146,你如果不会拍你就不用拍了，我想当地医生也跟你讲过我刚才已经跟你说过就是海南热血管瘤比较小的，...,1,C. Both informational and emotional support
238,好的，祝早日康复,1,D. Neither informational support or emotional ...
248,呃，没关系的，你那个线呢是应该是可吸收线其实我让你过来，到时候就是看看切口愈合的情况，这个线...,1,C. Both informational and emotional support
319,祝患者好运。,1,D. Neither informational support or emotional ...
408,放轻松！,1,B. Emotional Support
446,祝顺利康复,1,D. Neither informational support or emotional ...
447,1. 目前您是HPV感染导致的宫颈低级别病变，宫颈低级别鳞状上皮病变(LSIL，CIN1)这...,1,C. Both informational and emotional support
448,你的情况还可以，积极治疗没问题的,1,C. Both informational and emotional support


In [63]:
normalize_emotional_predictions = [1 if ( predict.startswith("B.") or predict.startswith("C.")) else 0 for predict in emotional_results]
mini_batch_emotional["normalize_predicted_label"] = normalize_emotional_predictions
mini_batch_emotional

,text,label,predicted_label,normalize_predicted_label
56,图片看上去是天疱疮，现在的口腔溃疡情况也是天疱疮引起的，口苦的症状是可能和药物有关，不要紧的,1,C. Both informational and emotional support,1
66,你做得很好。,1,D. Neither informational support or emotional ...,0
146,你如果不会拍你就不用拍了，我想当地医生也跟你讲过我刚才已经跟你说过就是海南热血管瘤比较小的，...,1,C. Both informational and emotional support,1
238,好的，祝早日康复,1,D. Neither informational support or emotional ...,0
248,呃，没关系的，你那个线呢是应该是可吸收线其实我让你过来，到时候就是看看切口愈合的情况，这个线...,1,C. Both informational and emotional support,1
319,祝患者好运。,1,D. Neither informational support or emotional ...,0
408,放轻松！,1,B. Emotional Support,1
446,祝顺利康复,1,D. Neither informational support or emotional ...,0
447,1. 目前您是HPV感染导致的宫颈低级别病变，宫颈低级别鳞状上皮病变(LSIL，CIN1)这...,1,C. Both informational and emotional support,1
448,你的情况还可以，积极治疗没问题的,1,C. Both informational and emotional support,1


In [57]:
mini_batch_non_emotional = emotional_data_train[emotional_data_train["label"] == 0].iloc[:21]
mini_batch_non_emotional["predicted_label"] = non_emotional_results
mini_batch_non_emotional

,text,label,predicted_label
0,他克莫司软膏（普特彼） 0.1%*10g 数量：1盒,0,D. Neither informational support or emotional ...
1,家附近医院心内科先做好,0,C. Both informational and emotional support
2,出院小结能否拍照给我？,0,D. Neither informational support or emotional ...
3,呃，有可能的，这个小朋友就这几个最常见的突然会加重的因素，脸上加重的话，这个食物如果身上还好...,0,C. Both informational and emotional support
4,我倾向良性概率大,0,"The physician's reply translates to ""I tend to..."
5,结节性质不能确定，建议右肺穿刺活检明确诊断；如果为肿瘤，做PET/CT评估疾病程度,0,C. Both informational and emotional support
6,性价比很高啊。,0,D. Neither informational support or emotional ...
7,没关系的。,0,D. Neither informational support or emotional ...
8,对的，要巩固,0,D. Neither informational support or emotional ...
9,好的，不客气,0,D. Neither informational support or emotional ...


In [65]:
normalize_non_emotional_predictions = [1 if ( predict.startswith("B.") or predict.startswith("C.")) else 0 for predict in non_emotional_results]
mini_batch_non_emotional["normalize_predicted_label"] = normalize_non_emotional_predictions
mini_batch_non_emotional

,text,label,predicted_label,normalize_predicted_label
0,他克莫司软膏（普特彼） 0.1%*10g 数量：1盒,0,D. Neither informational support or emotional ...,0
1,家附近医院心内科先做好,0,C. Both informational and emotional support,1
2,出院小结能否拍照给我？,0,D. Neither informational support or emotional ...,0
3,呃，有可能的，这个小朋友就这几个最常见的突然会加重的因素，脸上加重的话，这个食物如果身上还好...,0,C. Both informational and emotional support,1
4,我倾向良性概率大,0,"The physician's reply translates to ""I tend to...",0
5,结节性质不能确定，建议右肺穿刺活检明确诊断；如果为肿瘤，做PET/CT评估疾病程度,0,C. Both informational and emotional support,1
6,性价比很高啊。,0,D. Neither informational support or emotional ...,0
7,没关系的。,0,D. Neither informational support or emotional ...,0
8,对的，要巩固,0,D. Neither informational support or emotional ...,0
9,好的，不客气,0,D. Neither informational support or emotional ...,0


### Evaluation

In [71]:
merged_mini_batch_informational = pd.concat([mini_batch_informational, mini_batch_non_informational], axis=0)
merged_mini_batch_informational

,text,label,predicted_label,normalize_predicted_label
0,他克莫司软膏（普特彼） 0.1%*10g 数量：1盒,1,D. Neither informational support or emotional ...,0
3,呃，有可能的，这个小朋友就这几个最常见的突然会加重的因素，脸上加重的话，这个食物如果身上还好...,1,C. Both informational and emotional support,1
4,我倾向良性概率大,1,D. Neither informational support or emotional ...,0
5,结节性质不能确定，建议右肺穿刺活检明确诊断；如果为肿瘤，做PET/CT评估疾病程度,1,C. Both informational and emotional support,1
11,卵巢功能LH/FSH略高，睾酮不高，主要的治疗方法是减轻体重，控制好体重，内分泌就会改善,1,C. Both informational and emotional support,1
13,也就是说没有一个检查办法能够确诊，药物过敏。,1,D. Neither informational support or emotional ...,0
14,你心理负担比较重，那就做一下吧，应该没有什么问题的，做完后把结果给我看一下吧,1,C. Both informational and emotional support,1
16,建议阴道塞药，目前治疗HPV感染的药物很多。,1,A. Informational Support,1
17,这个是大头痱子,1,D. Neither informational support or emotional ...,0
18,他也是高的啊，那么我觉得这个还是要考虑这个呃，她是一个标志物啊，也就是肿瘤有进展的这个可能性...,1,C. Both informational and emotional support,1


In [75]:
# Informational Data

y_val = merged_mini_batch_informational["label"]
predicted = merged_mini_batch_informational["normalize_predicted_label"]

print("Informational Data Accuracy:", metrics.accuracy_score(y_val, predicted))
print("Informational Data Precision:", metrics.precision_score(y_val, predicted))
print("Informational Data Recall:", metrics.recall_score(y_val, predicted))
print("Informational Data F1_score:", metrics.f1_score(y_val, predicted))

Informational Data Accuracy: 0.7619047619047619
Informational Data Precision: 0.7619047619047619
Informational Data Recall: 0.7619047619047619
Informational Data F1_score: 0.7619047619047619


In [73]:
merged_mini_batch_emotional = pd.concat([mini_batch_emotional, mini_batch_non_emotional], axis=0)
merged_mini_batch_emotional

,text,label,predicted_label,normalize_predicted_label
56,图片看上去是天疱疮，现在的口腔溃疡情况也是天疱疮引起的，口苦的症状是可能和药物有关，不要紧的,1,C. Both informational and emotional support,1
66,你做得很好。,1,D. Neither informational support or emotional ...,0
146,你如果不会拍你就不用拍了，我想当地医生也跟你讲过我刚才已经跟你说过就是海南热血管瘤比较小的，...,1,C. Both informational and emotional support,1
238,好的，祝早日康复,1,D. Neither informational support or emotional ...,0
248,呃，没关系的，你那个线呢是应该是可吸收线其实我让你过来，到时候就是看看切口愈合的情况，这个线...,1,C. Both informational and emotional support,1
319,祝患者好运。,1,D. Neither informational support or emotional ...,0
408,放轻松！,1,B. Emotional Support,1
446,祝顺利康复,1,D. Neither informational support or emotional ...,0
447,1. 目前您是HPV感染导致的宫颈低级别病变，宫颈低级别鳞状上皮病变(LSIL，CIN1)这...,1,C. Both informational and emotional support,1
448,你的情况还可以，积极治疗没问题的,1,C. Both informational and emotional support,1


In [74]:
# Emotional Data

y_val = merged_mini_batch_emotional["label"]
predicted = merged_mini_batch_emotional["normalize_predicted_label"]

print("Emotional Data Accuracy:", metrics.accuracy_score(y_val, predicted))
print("Emotional Data Precision:", metrics.precision_score(y_val, predicted))
print("Emotional Data Recall:", metrics.recall_score(y_val, predicted))
print("Emotional Data F1_score:", metrics.f1_score(y_val, predicted))

Emotional Data Accuracy: 0.6428571428571429
Emotional Data Precision: 0.65
Emotional Data Recall: 0.6190476190476191
Emotional Data F1_score: 0.6341463414634146


## Zero-shot learning - Qwen2-7B - Separated prompts

In [2]:
informational_data_train = pd.read_csv("C:\SMU\LLM\data\informatical_train.csv", encoding="utf-8")
emotional_data_train = pd.read_csv("C:\SMU\LLM\data\emotional_train.csv", encoding="utf-8")

In [3]:
device = "cpu" # the device to load the model onto

# Now you do not need to add "trust_remote_code=True"
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-7B-Instruct",
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-7B-Instruct")

Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 17.74it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
# loop through 21 samples (0-20)
informational_results = []
for i, row in informational_data_train[informational_data_train["label"] == 1].iloc[:21].iterrows():
    informational_sample = row["text"]
    prompt = f"""Act as a sophisticated AI trained in the nuances of online communication, particularly in the healthcare domain. Given a physician's online reply, I need you to analyze the content and context of the message. Determine whether the reply constitutes informational support or not. Informational support should be considered as providing advice, facts, or guidance that can be used to solve a problem or address a question.
    
    Based on your assessment, return '1' for True if the reply offers informational support, and '0' for False if it does not.

    The online physician's reply for analysis:
    {informational_sample}
    """
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    # Directly use generate() and tokenizer.decode() to get the output.
    # Use `max_new_tokens` to control the maximum output length.
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    informational_results.append(response)

In [6]:
# loop through 21 samples (0-20)
non_informational_results = []
for i, row in informational_data_train[informational_data_train["label"] == 0].iloc[:21].iterrows():
    non_informational_sample = row["text"]
    prompt = f"""Act as a sophisticated AI trained in the nuances of online communication, particularly in the healthcare domain. Given a physician's online reply, I need you to analyze the content and context of the message. Determine whether the reply constitutes informational support or not. Informational support should be considered as providing advice, facts, or guidance that can be used to solve a problem or address a question.
    
    Based on your assessment, return '1' for True if the reply offers informational support, and '0' for False if it does not.

    The online physician's reply for analysis:
    {non_informational_sample}
    """
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    # Directly use generate() and tokenizer.decode() to get the output.
    # Use `max_new_tokens` to control the maximum output length.
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    non_informational_results.append(response)

In [7]:
# loop through 21 samples (0-20)
emotional_results = []
for i, row in emotional_data_train[emotional_data_train["label"] == 1].iloc[:21].iterrows():
    emotional_sample = row["text"]
    prompt = f"""Act as a discerning analyzer with expertise in both medical communication and emotional intelligence. Given a physician's online reply, I need you to analyze the content and context of the message. Based on the language, tone, and context used, decide whether the reply is offering emotional support. Emotional support should be considered as responses that offer empathy, concern, encouragement, reassurance, or aim to share the patient's happiness or sadness and to express caring and concern. 
    
    Based on your assessment, return '1' for True if the reply constitutes emotional support, and '0' for False if it does not.
    ONLY RETURN 1 or 0, NO MORE COMMENT.

    The online physician's reply for analysis:
    {emotional_sample}
    """
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    # Directly use generate() and tokenizer.decode() to get the output.
    # Use `max_new_tokens` to control the maximum output length.
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    emotional_results.append(response)

In [8]:
# loop through 21 samples (0-20)
non_emotional_results = []
for i, row in emotional_data_train[emotional_data_train["label"] == 0].iloc[:21].iterrows():
    non_emotional_sample = row["text"]
    prompt = f"""Act as a discerning analyzer with expertise in both medical communication and emotional intelligence. Given a physician's online reply, I need you to analyze the content and context of the message. Based on the language, tone, and context used, decide whether the reply is offering emotional support. Emotional support should be considered as responses that offer empathy, concern, encouragement, reassurance, or aim to share the patient's happiness or sadness and to express caring and concern. 
    
    Based on your assessment, return '1' for True if the reply constitutes emotional support, and '0' for False if it does not.
    ONLY RETURN 1 or 0, NO MORE COMMENT.
    
    The online physician's reply for analysis:
    {non_emotional_sample}
    """
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    # Directly use generate() and tokenizer.decode() to get the output.
    # Use `max_new_tokens` to control the maximum output length.
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    non_emotional_results.append(response)

In [21]:
mini_batch_informational = informational_data_train[informational_data_train["label"] == 1].iloc[:21]
mini_batch_informational["predicted_label"] = informational_results
normalize_informational_predictions = [int(predict) if (predict == "0" or predict == "1") else 0 for predict in informational_results]
mini_batch_informational["normalize_predicted_label"] = normalize_informational_predictions
mini_batch_informational

,text,label,predicted_label,normalize_predicted_label
0,他克莫司软膏（普特彼） 0.1%*10g 数量：1盒,1,0,0
3,呃，有可能的，这个小朋友就这几个最常见的突然会加重的因素，脸上加重的话，这个食物如果身上还好...,1,1,1
4,我倾向良性概率大,1,0,0
5,结节性质不能确定，建议右肺穿刺活检明确诊断；如果为肿瘤，做PET/CT评估疾病程度,1,The physician's reply is offering informationa...,0
11,卵巢功能LH/FSH略高，睾酮不高，主要的治疗方法是减轻体重，控制好体重，内分泌就会改善,1,1,1
13,也就是说没有一个检查办法能够确诊，药物过敏。,1,"To provide an accurate assessment, I would nee...",0
14,你心理负担比较重，那就做一下吧，应该没有什么问题的，做完后把结果给我看一下吧,1,"The given physician's reply is in Chinese, whi...",0
16,建议阴道塞药，目前治疗HPV感染的药物很多。,1,1,1
17,这个是大头痱子,1,0,0
18,他也是高的啊，那么我觉得这个还是要考虑这个呃，她是一个标志物啊，也就是肿瘤有进展的这个可能性...,1,"This Chinese sentence translates to: ""He is al...",0


In [22]:
mini_batch_non_informational = informational_data_train[informational_data_train["label"] == 0].iloc[:21]
mini_batch_non_informational["predicted_label"] = non_informational_results
normalize_non_informational_predictions = [int(predict) if (predict == "0" or predict == "1") else 0 for predict in non_informational_results]
mini_batch_non_informational["normalize_predicted_label"] = normalize_non_informational_predictions
mini_batch_non_informational

,text,label,predicted_label,normalize_predicted_label
1,家附近医院心内科先做好,0,The given text is in Chinese and translates to...,0
2,出院小结能否拍照给我？,0,0,0
6,性价比很高啊。,0,This reply seems to be in Chinese and translat...,0
7,没关系的。,0,0,0
8,对的，要巩固,0,"The given reply ""对的，要巩固"" translates from Chine...",0
9,好的，不客气,0,0,0
10,谈不上。,0,0,0
12,床位紧张，我们尽量吧,0,"The provided online physician's reply ""床位紧张，我们...",0
15,是的。,0,"To provide an accurate assessment, I would nee...",0
19,中医科医生知道的。,0,0,0


In [23]:
mini_batch_emotional = emotional_data_train[emotional_data_train["label"] == 1].iloc[:21]
mini_batch_emotional["predicted_label"] = emotional_results
normalize_emotional_predictions = [int(predict) if (predict == "0" or predict == "1") else 0 for predict in emotional_results]
mini_batch_emotional["normalize_predicted_label"] = normalize_emotional_predictions
mini_batch_emotional

,text,label,predicted_label,normalize_predicted_label
56,图片看上去是天疱疮，现在的口腔溃疡情况也是天疱疮引起的，口苦的症状是可能和药物有关，不要紧的,1,0,0
66,你做得很好。,1,"The given Chinese phrase ""你做得很好。"" translates t...",0
146,你如果不会拍你就不用拍了，我想当地医生也跟你讲过我刚才已经跟你说过就是海南热血管瘤比较小的，...,1,0,0
238,好的，祝早日康复,1,"The reply translates to ""Okay, wish you a spee...",0
248,呃，没关系的，你那个线呢是应该是可吸收线其实我让你过来，到时候就是看看切口愈合的情况，这个线...,1,This reply appears to be a mix of Chinese and ...,0
319,祝患者好运。,1,"The phrase ""祝患者好运"" translates to ""Wish the pat...",0
408,放轻松！,1,"The phrase ""放轻松"" translates to ""relax"" or ""tak...",0
446,祝顺利康复,1,"The Chinese phrase ""祝顺利康复"" translates to ""wish...",0
447,1. 目前您是HPV感染导致的宫颈低级别病变，宫颈低级别鳞状上皮病变(LSIL，CIN1)这...,1,The given reply from the physician does not pr...,0
448,你的情况还可以，积极治疗没问题的,1,"The provided Chinese message translates to ""Yo...",0


In [24]:
mini_batch_non_emotional = emotional_data_train[emotional_data_train["label"] == 0].iloc[:21]
mini_batch_non_emotional["predicted_label"] = non_emotional_results
normalize_non_emotional_predictions = [int(predict) if (predict == "0" or predict == "1") else 0 for predict in non_emotional_results]
mini_batch_non_emotional["normalize_predicted_label"] = normalize_non_emotional_predictions
mini_batch_non_emotional

,text,label,predicted_label,normalize_predicted_label
0,他克莫司软膏（普特彼） 0.1%*10g 数量：1盒,0,0,0
1,家附近医院心内科先做好,0,0,0
2,出院小结能否拍照给我？,0,"The given physician's reply translates to ""Can...",0
3,呃，有可能的，这个小朋友就这几个最常见的突然会加重的因素，脸上加重的话，这个食物如果身上还好...,0,0,0
4,我倾向良性概率大,0,"The provided text is in Chinese, which transla...",0
5,结节性质不能确定，建议右肺穿刺活检明确诊断；如果为肿瘤，做PET/CT评估疾病程度,0,The given text appears to be a medical advice ...,0
6,性价比很高啊。,0,This reply appears to be discussing the value ...,0
7,没关系的。,0,"The Chinese phrase ""没关系的"" translates to ""It's ...",0
8,对的，要巩固,0,0,0
9,好的，不客气,0,0,0


In [25]:
merged_mini_batch_informational = pd.concat([mini_batch_informational, mini_batch_non_informational], axis=0)
merged_mini_batch_informational

,text,label,predicted_label,normalize_predicted_label
0,他克莫司软膏（普特彼） 0.1%*10g 数量：1盒,1,0,0
3,呃，有可能的，这个小朋友就这几个最常见的突然会加重的因素，脸上加重的话，这个食物如果身上还好...,1,1,1
4,我倾向良性概率大,1,0,0
5,结节性质不能确定，建议右肺穿刺活检明确诊断；如果为肿瘤，做PET/CT评估疾病程度,1,The physician's reply is offering informationa...,0
11,卵巢功能LH/FSH略高，睾酮不高，主要的治疗方法是减轻体重，控制好体重，内分泌就会改善,1,1,1
13,也就是说没有一个检查办法能够确诊，药物过敏。,1,"To provide an accurate assessment, I would nee...",0
14,你心理负担比较重，那就做一下吧，应该没有什么问题的，做完后把结果给我看一下吧,1,"The given physician's reply is in Chinese, whi...",0
16,建议阴道塞药，目前治疗HPV感染的药物很多。,1,1,1
17,这个是大头痱子,1,0,0
18,他也是高的啊，那么我觉得这个还是要考虑这个呃，她是一个标志物啊，也就是肿瘤有进展的这个可能性...,1,"This Chinese sentence translates to: ""He is al...",0


In [26]:
merged_mini_batch_emotional = pd.concat([mini_batch_emotional, mini_batch_non_emotional], axis=0)
merged_mini_batch_emotional

,text,label,predicted_label,normalize_predicted_label
56,图片看上去是天疱疮，现在的口腔溃疡情况也是天疱疮引起的，口苦的症状是可能和药物有关，不要紧的,1,0,0
66,你做得很好。,1,"The given Chinese phrase ""你做得很好。"" translates t...",0
146,你如果不会拍你就不用拍了，我想当地医生也跟你讲过我刚才已经跟你说过就是海南热血管瘤比较小的，...,1,0,0
238,好的，祝早日康复,1,"The reply translates to ""Okay, wish you a spee...",0
248,呃，没关系的，你那个线呢是应该是可吸收线其实我让你过来，到时候就是看看切口愈合的情况，这个线...,1,This reply appears to be a mix of Chinese and ...,0
319,祝患者好运。,1,"The phrase ""祝患者好运"" translates to ""Wish the pat...",0
408,放轻松！,1,"The phrase ""放轻松"" translates to ""relax"" or ""tak...",0
446,祝顺利康复,1,"The Chinese phrase ""祝顺利康复"" translates to ""wish...",0
447,1. 目前您是HPV感染导致的宫颈低级别病变，宫颈低级别鳞状上皮病变(LSIL，CIN1)这...,1,The given reply from the physician does not pr...,0
448,你的情况还可以，积极治疗没问题的,1,"The provided Chinese message translates to ""Yo...",0


In [27]:
# Informational Data
y_val = merged_mini_batch_informational["label"]
predicted = merged_mini_batch_informational["normalize_predicted_label"]

print("Informational Data Accuracy:", metrics.accuracy_score(y_val, predicted))
print("Informational Data Precision:", metrics.precision_score(y_val, predicted))
print("Informational Data Recall:", metrics.recall_score(y_val, predicted))
print("Informational Data F1_score:", metrics.f1_score(y_val, predicted))

Informational Data Accuracy: 0.7619047619047619
Informational Data Precision: 0.9230769230769231
Informational Data Recall: 0.5714285714285714
Informational Data F1_score: 0.7058823529411765


In [29]:
# Emotional Data
y_val = merged_mini_batch_emotional["label"]
predicted = merged_mini_batch_emotional["normalize_predicted_label"]

print("Emotional Data Accuracy:", metrics.accuracy_score(y_val, predicted))
print("Emotional Data Precision:", metrics.precision_score(y_val, predicted))
print("Emotional Data Recall:", metrics.recall_score(y_val, predicted))
print("Emotional Data F1_score:", metrics.f1_score(y_val, predicted))

Emotional Data Accuracy: 0.5
Emotional Data Precision: 0.0
Emotional Data Recall: 0.0
Emotional Data F1_score: 0.0


c:\SMU\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Few-shot Learning - Qwen2-7B - Single prompt

In [30]:
# loop through 21 samples (0-20)
few_shot_informational_results = []
for i, row in informational_data_train[informational_data_train["label"] == 1].iloc[:21].iterrows():
    informational_sample = row["text"]
    prompt = f"""Act as a discerning analyzer with expertise in both medical communication and emotional intelligence. Given a physician's online reply, I need you to analyze the content and context of the message. Based on your analysis, choose the best tag in four tags below that describe it:
    A. Informational Support
    B. Emotional Support
    C. Both informational and emotional support
    D. Neither informational support or emotional support

    Informational support should be considered as providing advice, facts, or guidance that can be used to solve a problem or address a question.
    Emotional support should be considered as responses that offer empathy, concern, encouragement, reassurance, or aim to share the patient's happiness or sadness and to express caring and concern. 

    For a response to be tagged as both, it must clearly contain elements of both informational and emotional support. If the reply lacks elements of both types of support, it should be tagged as neither. 

    It's crucial that you focus solely on the content and the conveyed support type in the physician's reply to categorize it accurately. Please return only the chosen label. Here are some example for you:
    '''
    Input: 治疗费用一般一块一元硬币大小的面积1000元/次, 第一次多500元开机费。
    Output: A. Informational Support

    Input: 不排除这个可能 所以要经常拍片子 一般三个月拍一次,
    Output: A. Informational Support

    Input: 肿瘤标记物正常？
    Output: D. Neither informational support or emotional support

    Input: 带上所有片子，到我门诊看
    Output: D. Neither informational support or emotional support

    Input: 效果不好的话我们可以更换方案，不要着急。
    Output: B. Emotional Support

    Input: A, 不一定留疤的，好吧，你不要太紧张。
    Output: B. Emotional Support

    Input: 啊，你好，照片上和描述上我看我还是更倾向是个痔疮，因为尖锐湿疣一般是表面比较粗糙的一个疣状赘生物摸上去会比较硬，然后他很难拉平，但是痔疮的话一般是摸上去比较软，像肉球一样，因为他是一个血管静脉的一个APP，然后你拉一拉表面的皮是可以拉平的照片上看我还是更倾向是一个痔疮啊，不用过于担心。
    Output: C. Both informational and emotional support

    Input: 这不严重不严重，我也不建议你做任何的其他治疗定期复查观察，就可以了，没必要搞得很害怕的样子，没事儿的。
    Output: C. Both informational and emotional support
    '''

    The online physician's reply for analysis:
    {informational_sample}
    """
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    # Directly use generate() and tokenizer.decode() to get the output.
    # Use `max_new_tokens` to control the maximum output length.
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    few_shot_informational_results.append(response)

In [31]:
# loop through 21 samples (0-20)
few_shot_non_informational_results = []
for i, row in informational_data_train[informational_data_train["label"] == 0].iloc[:21].iterrows():
    non_informational_sample = row["text"]
    prompt = f"""Act as a discerning analyzer with expertise in both medical communication and emotional intelligence. Given a physician's online reply, I need you to analyze the content and context of the message. Based on your analysis, choose the best tag in four tags below that describe it:
    A. Informational Support
    B. Emotional Support
    C. Both informational and emotional support
    D. Neither informational support or emotional support

    Informational support should be considered as providing advice, facts, or guidance that can be used to solve a problem or address a question.
    Emotional support should be considered as responses that offer empathy, concern, encouragement, reassurance, or aim to share the patient's happiness or sadness and to express caring and concern. 

    For a response to be tagged as both, it must clearly contain elements of both informational and emotional support. If the reply lacks elements of both types of support, it should be tagged as neither. 

    It's crucial that you focus solely on the content and the conveyed support type in the physician's reply to categorize it accurately. Please return only the chosen label. Here are some example for you:
    '''
    Input: 治疗费用一般一块一元硬币大小的面积1000元/次, 第一次多500元开机费。
    Output: A. Informational Support

    Input: 不排除这个可能 所以要经常拍片子 一般三个月拍一次,
    Output: A. Informational Support

    Input: 肿瘤标记物正常？
    Output: D. Neither informational support or emotional support

    Input: 带上所有片子，到我门诊看
    Output: D. Neither informational support or emotional support

    Input: 效果不好的话我们可以更换方案，不要着急。
    Output: B. Emotional Support

    Input: A, 不一定留疤的，好吧，你不要太紧张。
    Output: B. Emotional Support

    Input: 啊，你好，照片上和描述上我看我还是更倾向是个痔疮，因为尖锐湿疣一般是表面比较粗糙的一个疣状赘生物摸上去会比较硬，然后他很难拉平，但是痔疮的话一般是摸上去比较软，像肉球一样，因为他是一个血管静脉的一个APP，然后你拉一拉表面的皮是可以拉平的照片上看我还是更倾向是一个痔疮啊，不用过于担心。
    Output: C. Both informational and emotional support

    Input: 这不严重不严重，我也不建议你做任何的其他治疗定期复查观察，就可以了，没必要搞得很害怕的样子，没事儿的。
    Output: C. Both informational and emotional support
    '''

    The online physician's reply for analysis:
    {non_informational_sample}
    """
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    # Directly use generate() and tokenizer.decode() to get the output.
    # Use `max_new_tokens` to control the maximum output length.
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    few_shot_non_informational_results.append(response)

In [32]:
# loop through 21 samples (0-20)
few_shot_emotional_results = []
for i, row in emotional_data_train[emotional_data_train["label"] == 1].iloc[:21].iterrows():
    emotional_sample = row["text"]
    prompt = f"""Act as a discerning analyzer with expertise in both medical communication and emotional intelligence. Given a physician's online reply, I need you to analyze the content and context of the message. Based on your analysis, choose the best tag in four tags below that describe it:
    A. Informational Support
    B. Emotional Support
    C. Both informational and emotional support
    D. Neither informational support or emotional support

    Informational support should be considered as providing advice, facts, or guidance that can be used to solve a problem or address a question.
    Emotional support should be considered as responses that offer empathy, concern, encouragement, reassurance, or aim to share the patient's happiness or sadness and to express caring and concern. 

    For a response to be tagged as both, it must clearly contain elements of both informational and emotional support. If the reply lacks elements of both types of support, it should be tagged as neither. 

    It's crucial that you focus solely on the content and the conveyed support type in the physician's reply to categorize it accurately. Please return only the chosen label. Here are some example for you:
    '''
    Input: 治疗费用一般一块一元硬币大小的面积1000元/次, 第一次多500元开机费。
    Output: A. Informational Support

    Input: 不排除这个可能 所以要经常拍片子 一般三个月拍一次,
    Output: A. Informational Support

    Input: 肿瘤标记物正常？
    Output: D. Neither informational support or emotional support

    Input: 带上所有片子，到我门诊看
    Output: D. Neither informational support or emotional support

    Input: 效果不好的话我们可以更换方案，不要着急。
    Output: B. Emotional Support

    Input: A, 不一定留疤的，好吧，你不要太紧张。
    Output: B. Emotional Support

    Input: 啊，你好，照片上和描述上我看我还是更倾向是个痔疮，因为尖锐湿疣一般是表面比较粗糙的一个疣状赘生物摸上去会比较硬，然后他很难拉平，但是痔疮的话一般是摸上去比较软，像肉球一样，因为他是一个血管静脉的一个APP，然后你拉一拉表面的皮是可以拉平的照片上看我还是更倾向是一个痔疮啊，不用过于担心。
    Output: C. Both informational and emotional support

    Input: 这不严重不严重，我也不建议你做任何的其他治疗定期复查观察，就可以了，没必要搞得很害怕的样子，没事儿的。
    Output: C. Both informational and emotional support
    '''

    The online physician's reply for analysis:
    {emotional_sample}
    """
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    # Directly use generate() and tokenizer.decode() to get the output.
    # Use `max_new_tokens` to control the maximum output length.
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    few_shot_emotional_results.append(response)

In [33]:
# loop through 21 samples (0-20)
few_shot_non_emotional_results = []
for i, row in emotional_data_train[emotional_data_train["label"] == 0].iloc[:21].iterrows():
    non_emotional_sample = row["text"]
    prompt = f"""Act as a discerning analyzer with expertise in both medical communication and emotional intelligence. Given a physician's online reply, I need you to analyze the content and context of the message. Based on your analysis, choose the best tag in four tags below that describe it:
    A. Informational Support
    B. Emotional Support
    C. Both informational and emotional support
    D. Neither informational support or emotional support

    Informational support should be considered as providing advice, facts, or guidance that can be used to solve a problem or address a question.
    Emotional support should be considered as responses that offer empathy, concern, encouragement, reassurance, or aim to share the patient's happiness or sadness and to express caring and concern. 

    For a response to be tagged as both, it must clearly contain elements of both informational and emotional support. If the reply lacks elements of both types of support, it should be tagged as neither. 

    It's crucial that you focus solely on the content and the conveyed support type in the physician's reply to categorize it accurately. Please return only the chosen label. Here are some example for you:
    '''
    Input: 治疗费用一般一块一元硬币大小的面积1000元/次, 第一次多500元开机费。
    Output: A. Informational Support

    Input: 不排除这个可能 所以要经常拍片子 一般三个月拍一次,
    Output: A. Informational Support

    Input: 肿瘤标记物正常？
    Output: D. Neither informational support or emotional support

    Input: 带上所有片子，到我门诊看
    Output: D. Neither informational support or emotional support

    Input: 效果不好的话我们可以更换方案，不要着急。
    Output: B. Emotional Support

    Input: A, 不一定留疤的，好吧，你不要太紧张。
    Output: B. Emotional Support

    Input: 啊，你好，照片上和描述上我看我还是更倾向是个痔疮，因为尖锐湿疣一般是表面比较粗糙的一个疣状赘生物摸上去会比较硬，然后他很难拉平，但是痔疮的话一般是摸上去比较软，像肉球一样，因为他是一个血管静脉的一个APP，然后你拉一拉表面的皮是可以拉平的照片上看我还是更倾向是一个痔疮啊，不用过于担心。
    Output: C. Both informational and emotional support

    Input: 这不严重不严重，我也不建议你做任何的其他治疗定期复查观察，就可以了，没必要搞得很害怕的样子，没事儿的。
    Output: C. Both informational and emotional support
    '''

    The online physician's reply for analysis:
    {non_emotional_sample}
    """
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    # Directly use generate() and tokenizer.decode() to get the output.
    # Use `max_new_tokens` to control the maximum output length.
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    few_shot_non_emotional_results.append(response)

In [37]:
few_shot_informational_results = [a.replace("Output: ", "") for a in few_shot_informational_results]
few_shot_informational_results

['A. Informational Support',
 'C. Both informational and emotional support',
 'C. Both informational and emotional support',
 'A. Informational Support',
 'A. Informational Support',
 'A. Informational Support',
 'B. Emotional Support',
 'A. Informational Support',
 'A. Informational Support',
 'A. Informational Support',
 'A. Informational Support',
 'A. Informational Support',
 'A. Informational Support',
 'C. Both informational and emotional support',
 'A. Informational Support',
 'A. Informational Support',
 'A. Informational Support',
 'A. Informational Support',
 'A. Informational Support',
 'A. Informational Support',
 'A. Informational Support']

In [39]:
few_shot_non_informational_results = [a.replace("Output: ", "") for a in few_shot_non_informational_results]
few_shot_non_informational_results

['A. Informational Support',
 'D. Neither informational support or emotional support',
 'D. Neither informational support or emotional support',
 'D. Neither informational support or emotional support',
 'A. Informational Support',
 'D. Neither informational support or emotional support',
 'D. Neither informational support or emotional support',
 'D. Neither informational support or emotional support',
 'D. Neither informational support or emotional support',
 'D. Neither informational support or emotional support',
 'A. Informational Support',
 'B. Emotional Support',
 'D. Neither informational support or emotional support',
 'A. Informational Support',
 'D. Neither informational support or emotional support',
 'A. Informational Support',
 'D. Neither informational support or emotional support',
 'D. Neither informational support or emotional support',
 'D. Neither informational support or emotional support',
 'C. Both informational and emotional support',
 'A. Informational Support']

In [41]:
few_shot_emotional_results = [a.replace("Output: ", "") for a in few_shot_emotional_results]
few_shot_emotional_results

['C. Both informational and emotional support',
 'B. Emotional Support',
 'C. Both informational and emotional support',
 'B. Emotional Support',
 'B. Emotional Support',
 'D. Neither informational support or emotional support',
 'B. Emotional Support',
 'D. Neither informational support or emotional support',
 'A. Informational Support',
 'C. Both informational and emotional support',
 'B. Emotional Support',
 'B. Emotional Support',
 'C. Both informational and emotional support',
 'B. Emotional Support',
 'B. Emotional Support',
 'C. Both informational and emotional support',
 'D. Neither informational support or emotional support',
 'A. Informational Support',
 'C. Both informational and emotional support',
 'A. Informational Support',
 'B. Emotional Support']

In [43]:
few_shot_non_emotional_results = [a.replace("Output: ", "") for a in few_shot_non_emotional_results]
few_shot_non_emotional_results

['A. Informational Support',
 'A. Informational Support',
 'D. Neither informational support or emotional support',
 'C. Both informational and emotional support',
 'B. Emotional Support',
 'A. Informational Support',
 'D. Neither informational support or emotional support',
 'D. Neither informational support or emotional support',
 'A. Informational Support',
 'D. Neither informational support or emotional support',
 'D. Neither informational support or emotional support',
 'A. Informational Support',
 'D. Neither informational support or emotional support',
 'A. Informational Support',
 'B. Emotional Support',
 'D. Neither informational support or emotional support',
 'A. Informational Support',
 'C. Both informational and emotional support',
 'A. Informational Support',
 'D. Neither informational support or emotional support',
 'A. Informational Support']

In [44]:
mini_batch_informational = informational_data_train[informational_data_train["label"] == 1].iloc[:21]
mini_batch_informational["predicted_label"] = few_shot_informational_results
normalize_informational_predictions = [1 if ( predict.startswith("A.") or predict.startswith("C.")) else 0 for predict in few_shot_informational_results]
mini_batch_informational["normalize_predicted_label"] = normalize_informational_predictions
mini_batch_informational

,text,label,predicted_label,normalize_predicted_label
0,他克莫司软膏（普特彼） 0.1%*10g 数量：1盒,1,A. Informational Support,1
3,呃，有可能的，这个小朋友就这几个最常见的突然会加重的因素，脸上加重的话，这个食物如果身上还好...,1,C. Both informational and emotional support,1
4,我倾向良性概率大,1,C. Both informational and emotional support,1
5,结节性质不能确定，建议右肺穿刺活检明确诊断；如果为肿瘤，做PET/CT评估疾病程度,1,A. Informational Support,1
11,卵巢功能LH/FSH略高，睾酮不高，主要的治疗方法是减轻体重，控制好体重，内分泌就会改善,1,A. Informational Support,1
13,也就是说没有一个检查办法能够确诊，药物过敏。,1,A. Informational Support,1
14,你心理负担比较重，那就做一下吧，应该没有什么问题的，做完后把结果给我看一下吧,1,B. Emotional Support,0
16,建议阴道塞药，目前治疗HPV感染的药物很多。,1,A. Informational Support,1
17,这个是大头痱子,1,A. Informational Support,1
18,他也是高的啊，那么我觉得这个还是要考虑这个呃，她是一个标志物啊，也就是肿瘤有进展的这个可能性...,1,A. Informational Support,1


In [45]:
mini_batch_non_informational = informational_data_train[informational_data_train["label"] == 0].iloc[:21]
mini_batch_non_informational["predicted_label"] = few_shot_non_informational_results
normalize_non_informational_predictions = [1 if ( predict.startswith("A.") or predict.startswith("C.")) else 0 for predict in few_shot_non_informational_results]
mini_batch_non_informational["normalize_predicted_label"] = normalize_non_informational_predictions
mini_batch_non_informational

,text,label,predicted_label,normalize_predicted_label
1,家附近医院心内科先做好,0,A. Informational Support,1
2,出院小结能否拍照给我？,0,D. Neither informational support or emotional ...,0
6,性价比很高啊。,0,D. Neither informational support or emotional ...,0
7,没关系的。,0,D. Neither informational support or emotional ...,0
8,对的，要巩固,0,A. Informational Support,1
9,好的，不客气,0,D. Neither informational support or emotional ...,0
10,谈不上。,0,D. Neither informational support or emotional ...,0
12,床位紧张，我们尽量吧,0,D. Neither informational support or emotional ...,0
15,是的。,0,D. Neither informational support or emotional ...,0
19,中医科医生知道的。,0,D. Neither informational support or emotional ...,0


In [46]:
mini_batch_emotional = emotional_data_train[emotional_data_train["label"] == 1].iloc[:21]
mini_batch_emotional["predicted_label"] = few_shot_emotional_results
normalize_emotional_predictions = [1 if ( predict.startswith("B.") or predict.startswith("C.")) else 0 for predict in few_shot_emotional_results]
mini_batch_emotional["normalize_predicted_label"] = normalize_emotional_predictions
mini_batch_emotional

,text,label,predicted_label,normalize_predicted_label
56,图片看上去是天疱疮，现在的口腔溃疡情况也是天疱疮引起的，口苦的症状是可能和药物有关，不要紧的,1,C. Both informational and emotional support,1
66,你做得很好。,1,B. Emotional Support,1
146,你如果不会拍你就不用拍了，我想当地医生也跟你讲过我刚才已经跟你说过就是海南热血管瘤比较小的，...,1,C. Both informational and emotional support,1
238,好的，祝早日康复,1,B. Emotional Support,1
248,呃，没关系的，你那个线呢是应该是可吸收线其实我让你过来，到时候就是看看切口愈合的情况，这个线...,1,B. Emotional Support,1
319,祝患者好运。,1,D. Neither informational support or emotional ...,0
408,放轻松！,1,B. Emotional Support,1
446,祝顺利康复,1,D. Neither informational support or emotional ...,0
447,1. 目前您是HPV感染导致的宫颈低级别病变，宫颈低级别鳞状上皮病变(LSIL，CIN1)这...,1,A. Informational Support,0
448,你的情况还可以，积极治疗没问题的,1,C. Both informational and emotional support,1


In [47]:
mini_batch_non_emotional = emotional_data_train[emotional_data_train["label"] == 0].iloc[:21]
mini_batch_non_emotional["predicted_label"] = few_shot_non_emotional_results
normalize_non_emotional_predictions = [1 if ( predict.startswith("B.") or predict.startswith("C.")) else 0 for predict in few_shot_non_emotional_results]
mini_batch_non_emotional["normalize_predicted_label"] = normalize_non_emotional_predictions
mini_batch_non_emotional

,text,label,predicted_label,normalize_predicted_label
0,他克莫司软膏（普特彼） 0.1%*10g 数量：1盒,0,A. Informational Support,0
1,家附近医院心内科先做好,0,A. Informational Support,0
2,出院小结能否拍照给我？,0,D. Neither informational support or emotional ...,0
3,呃，有可能的，这个小朋友就这几个最常见的突然会加重的因素，脸上加重的话，这个食物如果身上还好...,0,C. Both informational and emotional support,1
4,我倾向良性概率大,0,B. Emotional Support,1
5,结节性质不能确定，建议右肺穿刺活检明确诊断；如果为肿瘤，做PET/CT评估疾病程度,0,A. Informational Support,0
6,性价比很高啊。,0,D. Neither informational support or emotional ...,0
7,没关系的。,0,D. Neither informational support or emotional ...,0
8,对的，要巩固,0,A. Informational Support,0
9,好的，不客气,0,D. Neither informational support or emotional ...,0


In [49]:
merged_mini_batch_informational = pd.concat([mini_batch_informational, mini_batch_non_informational], axis=0)
merged_mini_batch_informational

,text,label,predicted_label,normalize_predicted_label
0,他克莫司软膏（普特彼） 0.1%*10g 数量：1盒,1,A. Informational Support,1
3,呃，有可能的，这个小朋友就这几个最常见的突然会加重的因素，脸上加重的话，这个食物如果身上还好...,1,C. Both informational and emotional support,1
4,我倾向良性概率大,1,C. Both informational and emotional support,1
5,结节性质不能确定，建议右肺穿刺活检明确诊断；如果为肿瘤，做PET/CT评估疾病程度,1,A. Informational Support,1
11,卵巢功能LH/FSH略高，睾酮不高，主要的治疗方法是减轻体重，控制好体重，内分泌就会改善,1,A. Informational Support,1
13,也就是说没有一个检查办法能够确诊，药物过敏。,1,A. Informational Support,1
14,你心理负担比较重，那就做一下吧，应该没有什么问题的，做完后把结果给我看一下吧,1,B. Emotional Support,0
16,建议阴道塞药，目前治疗HPV感染的药物很多。,1,A. Informational Support,1
17,这个是大头痱子,1,A. Informational Support,1
18,他也是高的啊，那么我觉得这个还是要考虑这个呃，她是一个标志物啊，也就是肿瘤有进展的这个可能性...,1,A. Informational Support,1


In [48]:
merged_mini_batch_emotional = pd.concat([mini_batch_emotional, mini_batch_non_emotional], axis=0)
merged_mini_batch_emotional

,text,label,predicted_label,normalize_predicted_label
56,图片看上去是天疱疮，现在的口腔溃疡情况也是天疱疮引起的，口苦的症状是可能和药物有关，不要紧的,1,C. Both informational and emotional support,1
66,你做得很好。,1,B. Emotional Support,1
146,你如果不会拍你就不用拍了，我想当地医生也跟你讲过我刚才已经跟你说过就是海南热血管瘤比较小的，...,1,C. Both informational and emotional support,1
238,好的，祝早日康复,1,B. Emotional Support,1
248,呃，没关系的，你那个线呢是应该是可吸收线其实我让你过来，到时候就是看看切口愈合的情况，这个线...,1,B. Emotional Support,1
319,祝患者好运。,1,D. Neither informational support or emotional ...,0
408,放轻松！,1,B. Emotional Support,1
446,祝顺利康复,1,D. Neither informational support or emotional ...,0
447,1. 目前您是HPV感染导致的宫颈低级别病变，宫颈低级别鳞状上皮病变(LSIL，CIN1)这...,1,A. Informational Support,0
448,你的情况还可以，积极治疗没问题的,1,C. Both informational and emotional support,1


In [50]:
# Informational Data
y_val = merged_mini_batch_informational["label"]
predicted = merged_mini_batch_informational["normalize_predicted_label"]

print("Informational Data Accuracy:", metrics.accuracy_score(y_val, predicted))
print("Informational Data Precision:", metrics.precision_score(y_val, predicted))
print("Informational Data Recall:", metrics.recall_score(y_val, predicted))
print("Informational Data F1_score:", metrics.f1_score(y_val, predicted))

Informational Data Accuracy: 0.8095238095238095
Informational Data Precision: 0.7407407407407407
Informational Data Recall: 0.9523809523809523
Informational Data F1_score: 0.8333333333333334


In [51]:
# Emotional Data
y_val = merged_mini_batch_emotional["label"]
predicted = merged_mini_batch_emotional["normalize_predicted_label"]

print("Emotional Data Accuracy:", metrics.accuracy_score(y_val, predicted))
print("Emotional Data Precision:", metrics.precision_score(y_val, predicted))
print("Emotional Data Recall:", metrics.recall_score(y_val, predicted))
print("Emotional Data F1_score:", metrics.f1_score(y_val, predicted))

Emotional Data Accuracy: 0.7619047619047619
Emotional Data Precision: 0.7894736842105263
Emotional Data Recall: 0.7142857142857143
Emotional Data F1_score: 0.75


# Inference

In [2]:
data = pd.read_csv("hdf_service_20230117_sentences.csv", delimiter="\t")
data

,service_id,all_doctor_text
0,8508082035,['请你下周五早上到华山医院虹桥院区门诊找我加号'
1,8508082035,'好的 第1个问题，拔牙没有任何关系可以拔的 第2个问题，可以口服强的松每天早上6粒'
2,8508082035,'一个月左右如果病情比较稳定，就改为一天6粒一天4粒'
3,8508082035,'我的门诊时间： 周一下午和周五早上（华山医院虹桥院区门诊1楼7号房间） 周二上午（华山医院...
4,8508082035,'赵重波 通知：FcRn 单抗HBM 9161治疗全身型重症肌无力临床研究华山入组的8例受试...
...,...,...
3065544,8887571478,['明天早上不吃不喝，早上就给您手术']
3065545,8887571559,['您好'
3065546,8887571559,'根据您的检查结果目前诊断明确，建议手术治疗，手术范围至少右侧甲状腺切除加右侧气管旁和气管前...
3065547,8887571865,['胆囊癌，肝肾功能都不太好 如果不达标，病人生存＜半年']


In [3]:
device = "cuda" # the device to load the model onto

# Now you do not need to add "trust_remote_code=True"
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-7B-Instruct",
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-7B-Instruct")

/home/yueliu/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.66it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
from csv import writer
few_shot_results = []
for i, row in data.iloc[2943610:].iterrows():
    sample = row["all_doctor_text"]
    prompt = f"""Act as a discerning analyzer with expertise in both medical communication and emotional intelligence. Given a physician's online reply, I need you to analyze the content and context of the message. Based on your analysis, choose the best tag in four tags below that describe it:
    A. Informational Support
    B. Emotional Support
    C. Both informational and emotional support
    D. Neither informational support or emotional support

    Informational support should be considered as providing advice, facts, or guidance that can be used to solve a problem or address a question.
    Emotional support should be considered as responses that offer empathy, concern, encouragement, reassurance, or aim to share the patient's happiness or sadness and to express caring and concern. 

    For a response to be tagged as both, it must clearly contain elements of both informational and emotional support. If the reply lacks elements of both types of support, it should be tagged as neither. 

    It's crucial that you focus solely on the content and the conveyed support type in the physician's reply to categorize it accurately. Please return only the chosen label. Here are some example for you:
    '''
    Input: 治疗费用一般一块一元硬币大小的面积1000元/次, 第一次多500元开机费。
    Output: A. Informational Support

    Input: 不排除这个可能 所以要经常拍片子 一般三个月拍一次,
    Output: A. Informational Support

    Input: 肿瘤标记物正常？
    Output: D. Neither informational support or emotional support

    Input: 带上所有片子，到我门诊看
    Output: D. Neither informational support or emotional support

    Input: 效果不好的话我们可以更换方案，不要着急。
    Output: B. Emotional Support

    Input: A, 不一定留疤的，好吧，你不要太紧张。
    Output: B. Emotional Support

    Input: 啊，你好，照片上和描述上我看我还是更倾向是个痔疮，因为尖锐湿疣一般是表面比较粗糙的一个疣状赘生物摸上去会比较硬，然后他很难拉平，但是痔疮的话一般是摸上去比较软，像肉球一样，因为他是一个血管静脉的一个APP，然后你拉一拉表面的皮是可以拉平的照片上看我还是更倾向是一个痔疮啊，不用过于担心。
    Output: C. Both informational and emotional support

    Input: 这不严重不严重，我也不建议你做任何的其他治疗定期复查观察，就可以了，没必要搞得很害怕的样子，没事儿的。
    Output: C. Both informational and emotional support
    '''

    The online physician's reply for analysis:
    {sample}
    """
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    # Directly use generate() and tokenizer.decode() to get the output.
    # Use `max_new_tokens` to control the maximum output length.
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    few_shot_results.append(response)
    with open('result.csv','a+') as fd:
        writer_object = writer(fd)
        writer_object.writerow([response])
        fd.close()

The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
